Before starting, converting data for regions


data = pd.io.stata.read_stata('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Files from research/wm_replication_dataset_2020_09_25.dta')
data.to_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Files from research/region_info.csv')

In [2]:
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import time

# 0. Importing all data:
- info about regions
- femicides cases
- trends of country for what concerns democracy
- elections results per region

In [3]:
regions_info = pd.read_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Repo_File/turkey_femicides_ironhack/Sources/region_info.csv')

In [4]:
victims = pd.read_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Repo_File/turkey_femicides_ironhack/Sources/victim_level_data.csv')

In [5]:
demo_trends = pd.read_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Repo_File/turkey_femicides_ironhack/Sources/country_trends.csv')

In [6]:
elections = pd.read_csv('/Users/segre/OneDrive/Desktop/Final Project/Turkey/Repo_File/turkey_femicides_ironhack/Sources/elections_cityresults.csv')

# 1. Data Cleaning - Starting with the regions.

Columns data:
- Province Code
- Province Name
- all_wm_prov: women’s murders in Turkey (per 100,000 people) per year
- knows_wm_prov: number of women’s murders (again, per 100,000 people) across Turkish
provinces
- tot_pop
- ethnic_turkish
- ethnic_kurdish
- ln_gdppc_cons: e (logged) GDP per capita 
- fm_diff_educ: Gender equality in education attainment is calculated by subtracting the percentage of men who finished at least high school from the percentage of women who finished at least high school
- divore_rate: every observation, the average divorce
rate (i.e. number of divorces per 1000 people) in the last five years is calculated
- mosques: Religiosity is measured by the number of mosques (per 1000 people) in a province
- ln_casualty: civil war is measured by the (logged) number of war-related killings that occurred in a
province between 1984 and 2010
- ceasefire: It takes the value of 1 in years 2013 and 2014, and 0 otherwise. This variable is interacted with Civil War Exposure.

In [7]:
regions_info = regions_info.drop('Unnamed: 0', axis=1)

In [8]:
regions_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   prov_code         2460 non-null   int64  
 1   prov_name         2460 non-null   object 
 2   year              2460 non-null   int64  
 3   all_wm_prov       648 non-null    float64
 4   knows_wm_prov     648 non-null    float64
 5   total_pop         648 non-null    float64
 6   ethnic_turkish    648 non-null    float64
 7   ethnic_kurdish    648 non-null    float64
 8   ln_casualty       648 non-null    float64
 9   ln_gdppc_cons     891 non-null    float64
 10  divorce_rate_ma5  1377 non-null   float64
 11  mosques_perk      648 non-null    float64
 12  ceasefire         2460 non-null   int64  
 13  fm_diff_educ      810 non-null    float64
dtypes: float64(10), int64(3), object(1)
memory usage: 269.2+ KB


We can already drop the years we do not need. Our dataset on femicides goes from 2010 to 2017. So for now, I will only keep data from 2009 to 2017.

In [9]:
regions_info.drop(regions_info[regions_info['year'] < 2010].index, inplace = True) 

In [10]:
regions_info.drop(regions_info[regions_info['year'] > 2017].index, inplace = True) 

In [11]:
regions_info['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype=int64)

We do not convert to datetime as we only have the years.

### Exploration of data regions

Check for duplicates

In [12]:
sum(regions_info.duplicated())

0

Check for nulls. Nothing needed.

In [13]:
round(regions_info.isna().sum()/len(regions_info),4)*100

prov_code           0.0
prov_name           0.0
year                0.0
all_wm_prov         0.0
knows_wm_prov       0.0
total_pop           0.0
ethnic_turkish      0.0
ethnic_kurdish      0.0
ln_casualty         0.0
ln_gdppc_cons       0.0
divorce_rate_ma5    0.0
mosques_perk        0.0
ceasefire           0.0
fm_diff_educ        0.0
dtype: float64

Final clean-up

In [14]:
regions_info['prov_name'] = regions_info['prov_name'].str.lower()

Check data details

In [15]:
regions_info.describe(include = 'all')

,prov_code,prov_name,year,all_wm_prov,knows_wm_prov,total_pop,ethnic_turkish,ethnic_kurdish,ln_casualty,ln_gdppc_cons,divorce_rate_ma5,mosques_perk,ceasefire,fm_diff_educ
count,648.000000,648,648.000000,648.000000,648.000000,6.480000e+02,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000
unique,NaN,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,amasya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,41.000000,NaN,2013.500000,3.030864,2.915123,9.534026e+05,75.425493,21.467349,2.345763,4.344408,1.283133,1.843475,0.250000,-0.100884
std,23.398966,NaN,2.293058,5.068218,4.868333,1.706082e+06,33.867957,32.434790,2.623615,0.342109,0.616719,1.160641,0.433347,0.028900
min,1.000000,NaN,2010.000000,0.000000,0.000000,7.441200e+04,0.000000,0.000000,0.000000,3.463987,0.114000,0.226302,0.000000,-0.184529
25%,21.000000,NaN,2011.750000,0.000000,0.000000,2.808055e+05,62.074455,0.000000,0.000000,4.108124,0.857000,1.120346,0.000000,-0.119214
50%,41.000000,NaN,2013.500000,1.000000,1.000000,5.131070e+05,94.709680,3.169473,1.098612,4.336241,1.342000,1.577232,0.000000,-0.099548
75%,61.000000,NaN,2015.250000,4.000000,3.000000,9.948722e+05,99.407780,25.884853,4.204693,4.566989,1.709000,2.132194,0.250000,-0.080654


# 2a. Data Cleaning and Translations - Victims Cases

In [16]:
victims.rename(columns = {'isim':'victim_name', 'yaş':'age', 'tarih':'date',
                         'yer':'place','fail':'perpetrator',
                         'bahane':'excuse', 'hikaye':'story' }, inplace = True) 


In [17]:
victims1 = victims.drop('link', axis=1)

In [18]:
victims1 = victims1.drop('victim_name', axis=1)

Splitting locations to have a better overview on cities

In [19]:
victims2 = victims1.join(victims['place'].str.split(',', 1, expand=True).rename(columns={0:'district', 1:'prov_name'}))
#city is enough, we can drop district (too granular)

In [20]:
victims2 = victims2.drop('district', axis=1)

In [21]:
victims2['prov_name'] = victims2['prov_name'].str.lower()

In [22]:
victims2 = victims2.drop('place', axis=1)

In [23]:
victims2

,age,date,perpetrator,excuse,story,prov_name
0,18.0,31/12/2017,Erkek arkadaşı,Bilinmiyor,"Erhan M. adlı kişi, sevgilisi Buket Şişek'i 5....",istanbul
1,17.0,31/12/2017,Erkek arkadaşı,Kadının ayrılma isteği,"19 yaşındaki Furkan K., kendisinden ayrılmak i...",burdur
2,38.0,30/12/2017,Erkek arkadaşı,Tartışma/ kavga,"Tevfik K. adlı kişi, tartışma üzerine sevgilis...",bolu
3,53.0,27/12/2017,Erkek arkadaşı,Kadının ayrılma isteği,"Seyfali Akkoç adlı kişi, bir süredir kendisind...",ankara
4,22.0,25/12/2017,Kocası,Aldatılma şüphesi,Şeyhmus İ. adlı kişi kendisini aldattığı iddia...,diyarbakir
...,...,...,...,...,...,...
1959,24.0,14/01/2010,Akrabası,Erkeğin ayrılma isteğinin reddedilmesi,"İddiaya göre Yunus F., eşinin kardeşinin karıs...",ankara
1960,34.0,12/1/2010,Saldırgan,Erkeğin reddedilmesi/ terk edilmesi,Cezaevindeki iyi hali nedeniyle Adana Adliyesi...,adana
1961,57.0,12/1/2010,Erkek kardeşi,Bilinmiyor,"48 yaşındaki Yusuf D., evine gelen ablası Alim...",kutahya
1962,34.0,11/1/2010,Erkek arkadaşı,Bilinmiyor,"26 yaşındaki Uzman Onbaşı Harun Ceylan, sevgil...",adana


Translate from Turkish 

In [24]:
victims2['perpetrator'].unique()

array(['Erkek arkadaşı', 'Kocası', 'Oğlu', 'Eski kocası',
       'Tanıdığı bir erkek', 'Babası', 'Diğer yakını',
       'Dini nikahlı kocası', 'Üvey oğlu', 'Erkek kardeşi',
       'Eski erkek arkadaşı', 'Akrabası', 'Damadı', 'Saldırgan',
       'Dini nikahlı eski kocası', 'Kayınpederi'], dtype=object)

In [25]:
translations = {
        'Erkek arkadaşı':'boyfriend',
        'Kocası':'husband',
        'Oğlu':'son',
        'Eski kocası':'ex-husband',
        'Tanıdığı bir erkek':'known man',
        'Babası':'father',
        'Diğer yakını':'other relative',
        'Dini nikahlı kocası':'religiously-married husband',
        'Üvey oğlu':'step-son',
        'Erkek kardeşi':'brother',
        'Eski erkek arkadaşı':'ex-boyfriend',
        'Akrabası':'relative',
        'Damadı':'fiancè',
        'Saldırgan':'attacker', 
        'Dini nikahlı eski kocası':'religiously married ex-husband', 
        'Kayınpederi':'father-in-law'
    }

victims2['perpetrator_en'] = victims2['perpetrator'].replace(translations)

Remapping perpetratore categories

In [26]:
remapping_perpetrator = {
        'boyfriend': ['boyfriend'],
        'partner':['fiancè','husband','religiously-married husband'],
        'son':['son', 'step-son'],
        'known man': ['known man'],
        'father':['father'],
        'relative':['other relative','relative'],
        'brother':['brother'],
        'attacker':['attacker'],
        'ex-partner':['ex-husband','religiously married ex-husband', 'ex-boyfriend'],
        'father-in-law':['father-in-law']
    }
victims2['perpetrator_en_new'] = victims2['perpetrator_en'].map(lambda x: next((k for k, v in remapping_perpetrator.items() if x in v), None))

In [27]:
victims2 = victims2.drop('perpetrator', axis=1)


In [28]:
victims2 =victims2.drop('perpetrator_en', axis=1)

In [29]:
victims2.rename(columns={'perpetrator_en_new': 'perp_cat'})

,age,date,excuse,story,prov_name,perp_cat
0,18.0,31/12/2017,Bilinmiyor,"Erhan M. adlı kişi, sevgilisi Buket Şişek'i 5....",istanbul,boyfriend
1,17.0,31/12/2017,Kadının ayrılma isteği,"19 yaşındaki Furkan K., kendisinden ayrılmak i...",burdur,boyfriend
2,38.0,30/12/2017,Tartışma/ kavga,"Tevfik K. adlı kişi, tartışma üzerine sevgilis...",bolu,boyfriend
3,53.0,27/12/2017,Kadının ayrılma isteği,"Seyfali Akkoç adlı kişi, bir süredir kendisind...",ankara,boyfriend
4,22.0,25/12/2017,Aldatılma şüphesi,Şeyhmus İ. adlı kişi kendisini aldattığı iddia...,diyarbakir,partner
...,...,...,...,...,...,...
1959,24.0,14/01/2010,Erkeğin ayrılma isteğinin reddedilmesi,"İddiaya göre Yunus F., eşinin kardeşinin karıs...",ankara,relative
1960,34.0,12/1/2010,Erkeğin reddedilmesi/ terk edilmesi,Cezaevindeki iyi hali nedeniyle Adana Adliyesi...,adana,attacker
1961,57.0,12/1/2010,Bilinmiyor,"48 yaşındaki Yusuf D., evine gelen ablası Alim...",kutahya,brother
1962,34.0,11/1/2010,Bilinmiyor,"26 yaşındaki Uzman Onbaşı Harun Ceylan, sevgil...",adana,boyfriend


Decided to drop the stories altogether

In [30]:
victims2= victims2.drop('story', axis=1)

Translating the excuse

In [31]:
#victims2['excuse'].unique()

In [32]:
#from google_trans_new import google_translator  
#translator = google_translator()  
#translations = {}
#for column in victims2.columns:
    # unique elements of the column
 #   unique_elements = victims2[column].unique()
  #  for element in unique_elements:
        # add translation to the dictionary
         #time.sleep(10)
      #  translations[element] = translator.translate(element)
    
#print(translations[5])

Getting a too many requests error from google translate API, translating the excuse with a simple map

In [33]:
translations_excuses= {
        'Bilinmiyor':'Unknown',
'Kadının ayrılma isteği':'The womans desire to leave',
'Tartışma/ kavga':' Argument / fight',
'Aldatılma şüphesi':'Suspicion of being deceived',
'Erkeğin barışma isteğinin reddedilmesi':'Refusing the mans request for reconciliation',
'Maddi kaynaklı':'Financially sourced',
'Diğer, Erkeğin "erkekliği"':'Other, the man s manhood',
'Diğer, Psikolojik sorun':'Other Psychological problem',
'Erkeğin reddedilmesi/ terk edilmesi':'The rejection / abandonment of the man',
'Kaza iddiası':'Accident claim',
'Dolaylı':'Indirect',
'Kadının karar ve tercihlerine ilişkin, Kadının seks işçiliğine karşı direnmesi':'Regarding the decisions and preferences of women womens resistance to sex work',
'Kadının boşanma isteği':'The womans desire for divorce',
'Namus/ töre':'Honor',
'Kadının karar ve tercihlerine ilişkin, Kadının dedikodu yapması':'The womans gossip about the womans decisions and preferences',
'Kıskançlık':'Jealousy',
'Cinsel saldırı':'Sexual assault',
'Diğer, Başka bir olaydan kadını sorumlu tutma':'Other holding a woman responsible for another incident',
'Kadının karar ve tercihlerine ilişkin, Kadının hakaret etmesi':'Regarding the womans decisions and preferences the womans insulting',
'Kadının karar ve tercihlerine ilişkin, Kadının erkekten içki içmemesini istemesi':'Regarding the womans decisions and preferences the womans asking the man not to drink alcohol',
'Kadının karar ve tercihlerine ilişkin, Kadının sosyal medya hesabı':'Womens social media account regarding the decisions and preferences of the woman',
'Diğer, Yönlendirme':'Other Routing',
'Çocuk/ velayet kaynaklı':'Child / custody sourced',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeğin ilişkisini onaylamaması':'Regarding the womans decisions and preferences the womans disapproval of the mans relationship',
'Kadının karar ve tercihlerine ilişkin, Kadının evden ayrılmak istemesi':'Regarding the womans decisions and preferences the womans willingness to leave the house',
'Hırsızlık':'Theft',
'Erkeğin hizmet beklentisi, Kadının yemeğe salça koyması':'Male service expectation Woman putting tomato paste in food',
'Erkeğin ayrılma isteğinin reddedilmesi':'Refusal of a mans request to leave',
'Kadının karar ve tercihlerine ilişkin, Erkeğin kadının işini onaylamaması':'Regarding the womans decisions and preferences the mans disapproval of the womans job',
'Kadının karar ve tercihlerine ilişkin, Erkeğin kadının ilişkisini onaylamaması':'Regarding the womans decisions and preferences the mans disapproval of the womans relationship',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeğin tacizini ifşa etmesi':'Regarding the decisions and preferences of the woman the womans disclosure of the harassment of the man',
'Diğer, Erkeğe vahiy gelmesi':'The other is the coming of revelation to the man',
'Diğer, Erkeğe cinayeti cinlerin söylemesi':'Other the murder of the man is told by the jinn',
'Cinsiyet kimliği':'Gender identity',
'Kadının karar ve tercihlerine ilişkin, Kadının telefon şifresini söylememesi':'Regarding the womans decisions and preferences the womans not saying her phone password',
'Kadının karar ve tercihlerine ilişkin, Kadının abisinde kalması':'Regarding the decisions and preferences of the woman the woman should remain in her brother',
'Diğer, Erkeğin sinirlerine hakim olamaması':'The other is that the man cannot control his nerves',
'Diğer, Erkeğin uyuşturucunun etkisine girmesi':'The other is that the man is under the influence of drugs',
'Kadının karar ve tercihlerine ilişkin, Kadının ailesinde kalmak istemesi':'Regarding the womans decisions and preferences the womans willingness to stay in her family',
'Diğer, Erkeğin cinayetle suçlanması':'The other is that the man is accused of murder',
'Kadının karar ve tercihlerine ilişkin, Kadının kardeşini koruması':'Regarding the womans decisions and preferences Womans protection of her brother',
'Diğer, Kadının uykusunda erkeğe saldırması':'Other a woman attacking a man in her sleep',
'Kadının karar ve tercihlerine ilişkin, Kadının telefonda çok konuşması':'Women talking a lot on the phone regarding the decisions and preferences of the woman',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeği şikayet etmesi':'Regarding the womans decisions and preferences Womans complaint against the man',
'Kadının karar ve tercihlerine ilişkin, Kadının boşandığı eşine dönmesi':'Regarding the womans decisions and preferences the womans return to her divorced spouse',
'Kadının karar ve tercihlerine ilişkin, Kadının sık sevgili değiştirmesi':'Regarding the womans decisions and preferences the womans frequent change of lovers',
'Kadının karar ve tercihlerine ilişkin, Kadının geç saatlere kadar gezmesi':'Regarding the womans decisions and preferences Womans traveling until late hours',
'Diğer, Erkeğin bunalması':'Other the man is overwhelmed',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeği rezil edeceğini söylemesi':'Regarding the womans decisions and preferences the womans saying that she will disgrace the man',
'Diğer, Erkeğin gürültüden rahatsız olması':'The other is that the man is disturbed by the noise',
'Kadının karar ve tercihlerine ilişkin, Kadının yurttan kaçması':'Regarding the womans decisions and preferences Womans escape from the dormitory',
'Diğer, Erkeğin söylediği yalanın ortaya çıkması':'The other is the disclosure of the lie that the man told',
'Erkeğin hizmet beklentisi, Kadının su vermemesi':'Male service expectation Woman not giving water',
'Kadının karar ve tercihlerine ilişkin, Kadının kızına aldığı elbisenin etek boyu':'Regarding the decisions and preferences of the woman the skirt length of the dress the woman bought for her daughter',
'Diğer, Kadının telefonuna mesaj gelmesi':'Other a message on the womans phone',
'Erkeğin hizmet beklentisi, Kadının erkeğin derdini dinlememesi':'Male service expectation Woman not listening to mans troubles',
'Diğer, Cinsiyet':'Other Gender',
'Erkeğin hizmet beklentisi, Kadının yemek hazırlamaması':'Male service expectation Woman not preparing food',
'Kadının karar ve tercihlerine ilişkin, Kadının saçını kızıla boyatması':'Regarding the womans decisions and preferences the woman dying her hair red',
'Kadının karar ve tercihlerine ilişkin, Kadının yaptırdığı dövme':'A tattoo made by the woman regarding the decisions and preferences of the woman',
'Erkeğin hizmet beklentisi, Kadının çamaşır yıkamaması':'Male service expectation Woman not washing clothes',
'Erkeğin hizmet beklentisi, Kadının erkeğin annesini seyrek ziyaret etmesi':'Male service expectation Womans frequent visits to her mother',
'Kadının karar ve tercihlerine ilişkin, Kadının gülmesi':'Regarding the decisions and preferences of the woman the laugh of the woman',
'Kadının karar ve tercihlerine ilişkin, Kadının ülkesine dönme isteği':'Regarding the decisions and preferences of the woman the desire of the woman to return to her country',
'Diğer, Erkeğin kuma getirme isteğinin reddedilmesi':'Other refusal of the mans request to bring it to the sand',
'Diğer, Erkeğe evliliğin zor gelmesi':'Other it is difficult to get married to a man',
'Diğer, Kadının evlenme isteği':'Other the womans desire to marry',
'Kadının karar ve tercihlerine ilişkin, Kadının akrabalarıyla görüşmesi':'The womans meeting with her relatives regarding the womans decisions and preferences',
'Kadının karar ve tercihlerine ilişkin, Kadının erkeğin kardeşine yardım etmesi':'Regarding the womans decisions and preferences the woman helping the mans brother',
'Erkeğin hizmet beklentisi, Kadının erkek çocuk doğurmaması':'Male service expectation Woman not giving birth to male children',
'Kadının karar ve tercihlerine ilişkin, Kadının nafaka talebi':'Regarding the womans decisions and preferences the womans alimony request',
'Erkeğin hizmet beklentisi, Kadının erkeğe ilgi göstermemesi':'Male service expectation Woman not showing interest in men',
'Kadının karar ve tercihlerine ilişkin, Kadının otomobil istemesi':'Regarding the womans decisions and preferences the womans request for a car',
'Kadının karar ve tercihlerine ilişkin, Kadının elbise istemesi':'Regarding the womans decisions and preferences the womans request for a dress',
'Kadının karar ve tercihlerine ilişkin, Kadının çocuğu dövmesi':'Regarding the womans decisions and preferences Womans tattooing the child',
'Diğer, Kan davası':'Other Blood feud',
'Diğer, Erkeğin gıcık olması':'The other is that the man is annoyed',
'Kadının karar ve tercihlerine ilişkin, Kadının aleyhte tanıklık yapması':'The womans testimony against the womans decisions and preferences',
'Kadının karar ve tercihlerine ilişkin, Kadının eve geç gelmesi':'Regarding the womans decisions and preferences Woman coming home late',
'Erkeğin hizmet beklentisi, Kadının patates köfte yapmaması':'Male service expectation Woman not making potato meatballs',
'Diğer, Sebepsiz':'Other No Reason',
'Kadının karar ve tercihlerine ilişkin, Kadının küsmesi':'Regarding the decisions and preferences of the woman Woman being offended',
'Kadının karar ve tercihlerine ilişkin, Kadının yazlık ev istemesi':'Regarding the womans decisions and preferences the womans request for a summer house',
'Erkeğin hizmet beklentisi, Kadının çocuk sahibi olmaması':'Male service expectation Woman not having children',
'Erkeğin hizmet beklentisi, Kadının erkeğe iyi bakmaması':'Male service expectation Woman not taking good care of men',
'Erkeğin hizmet beklentisi, Kadının evi kirletmesi':'Male service expectation Woman polluting the house',
'Erkeğin boşanma isteğinin reddedilmesi':'The rejection of a mans divorce request',
'Kadının karar ve tercihlerine ilişkin, Kadının aracı yanlış yönlendirmesi':'Woman misleads the tool regarding the decisions and preferences of the woman',
'Diğer, Çocuğun yemek yememesi':'Other If the child does not eat',
'Erkeğin hizmet beklentisi, Kadının tuzluğu uzatmaması':'Male service expectation Woman not extending the salt shaker',
'Erkeğin kadının işini onaylamaması':'The man does not approve of the womans job',
'Erkeğin hizmet beklentisi, Kadının yemeği geç hazırlaması':'Male service expectation Woman preparing dinner late',
'Erkeğin hizmet beklentisi, Kadının ev işlerini yapmaması':'Male service expectation Woman not doing housework',
'Erkeğin hizmet beklentisi, Kadının sabah erkeği uyandırmaması':'Male service expectation Woman not waking up the man in the morning',
'Kadının karar ve tercihlerine ilişkin, Kadının yeni elbise alması':'Regarding the womans decisions and preferences the womans buying new clothes',
'Kadının karar ve tercihlerine ilişkin, Kadının başörtüsü takmak istememesi':'Regarding the womans decisions and preferences the woman does not want to wear a headscarf',
'Erkeğin hizmet beklentisi, Kadının koyunları otlatmaması':'Male service expectation woman not grazing sheep',
'Kadının karar ve tercihlerine ilişkin, Kadının oğlunda kalma isteği':'Regarding the womans decisions and preferences the womans desire to stay with her son',
 }

victims2['excuse_en'] = victims2['excuse'].replace(translations_excuses)

In [34]:
len(victims2['excuse_en'].unique())

98

# b. Remapping
After translations, I have realized there are too many categories for the excuses given for the femicides. I will cluster them in fewer buckets.

The main categories of excuses seem to be:
- Unknown
- Psychological issue
- Money Problems
- Woman quest for freedom (i.e. leaving the household, asking for divorce, etc.)
- Indirect
- Household expectations (i.e. not waking up the man, preparing dinner late, 
- Body expectations (i.e. not having children, 

In [35]:
remapping_excuses = {
        'Children':['Child / custody sourced',
                    'Other If the child does not eat',
                    'Regarding the womans decisions and preferences the womans desire to stay with her son'],
        'Fight':['Other holding a woman responsible for another incident'],
        'Household service expectations':['Male service expectation Woman putting tomato paste in food',
                                          'Male service expectation Woman not giving water',
                                          'Male service expectation Woman not preparing food',
                                          'Male service expectation',
                                          'Woman not washing clothes',
                                          'Male service expectation Woman not extending the salt shaker',
                                          'Male service expectation Woman preparing dinner late',
                                          'Male service expectation Woman not doing housework',
                                          'Male service expectation Woman not waking up the man in the morning',
                                         'Male service expectation woman not grazing sheep',
                                         'Male service expectation Woman not making potato meatballs',
                                         'Male service expectation Woman not washing clothes'],
        'Male service expectations':['Male service expectation Woman not listening to mans troubles',
                                     'Male service expectation Womans frequent visits to her mother',
                                     'Male service expectation Woman not giving birth to male children',
                                     'Male service expectation Woman not showing interest in men',
                                     'Male service expectation Woman not having children',
                                     'Male service expectation Woman polluting the house',
                                     'The man does not approve of the womans job',
                                    'The other is that the man is annoyed'],
        'Masculinity&Honor':['Other, the man s manhood',
                             'Honor',
                             'Jealousy',
                             'Refusal of a mans request to leave',
                             'Other the man is overwhelmed',
                             'Other a message on the womans phone',
                             'Male service expectation Woman not taking good care of men',
                            'The other is that the man cannot control his nerves'],
        'Other':[' Argument / fight','Financially sourced',
                 'Other Psychological problem',
                 'Accident claim',
                 'Other Routing',
                 'The other is the coming of revelation to the man',
                 'The other is that the man is under the influence of drugs',
                 'The other is that the man is accused of murder',
                 'The other is that the man is disturbed by the noise',
                 'The other is the disclosure of the lie that the man told',
                'Other Gender',
                 'Other refusal of the mans request to bring it to the sand',
                 'Other it is difficult to get married to a man',
                 'Other Blood feud',
                 'Other No Reason','Theft','Indirect'],
        'Separation & Jealousy':['Refusing the mans request for reconciliation',
                                 'The womans desire to leave',
                                 'Suspicion of being deceived',
                                 'The rejection / abandonment of the man',
                                 'Regarding the womans decisions and preferences the womans disapproval of the mans relationship',
                                 'Regarding the womans decisions and preferences the womans return to her divorced spouse',
                                 'The rejection of a mans divorce request',
                                'The womans desire for divorce',
                                'Regarding the womans decisions and preferences the womans willingness to leave the house'],
        'Sexual expectation':['Regarding the decisions and preferences of women womens resistance to sex work',
                              'Sexual assault',
                              'Other a woman attacking a man in her sleep'],
        'Unknown':['Unknown'],
        'Woman Decision and Preferences':['The womans gossip about the womans decisions and preferences',
                                        'Regarding the womans decisions and preferences the womans insulting',
                                        'Regarding the womans decisions and preferences the womans asking the man not to drink alcohol',
                                        'Womens social media account regarding the decisions and preferences of the woman',
                                        'Regarding the womans decisions and preferences the mans disapproval of the womans job',
                                        'Regarding the womans decisions and preferences the mans disapproval of the womans relationship',
                                        'Regarding the decisions and preferences of the woman the womans disclosure of the harassment of the man',
                                        'Other the murder of the man is told by the jinn',
                                        'Gender identity',
                                        'Regarding the womans decisions and preferences the womans not saying her phone password',
                                        'Regarding the decisions and preferences of the woman the woman should remain in her brother',
                                        'Regarding the womans decisions and preferences the womans willingness to stay in her family',
                                        'Regarding the womans decisions and preferences Womans protection of her brother',
                                        'Women talking a lot on the phone regarding the decisions and preferences of the woman',
                                        'Regarding the womans decisions and preferences Womans complaint against the man',
                                        'Regarding the womans decisions and preferences the womans frequent change of lovers',
                                        'Regarding the womans decisions and preferences Womans traveling until late hours',
                                        'Regarding the womans decisions and preferences the womans saying that she will disgrace the man',
                                        'Regarding the womans decisions and preferences Womans escape from the dormitory',
                                        'Regarding the decisions and preferences of the woman the skirt length of the dress the woman bought for her daughter',
                                        'Regarding the womans decisions and preferences the woman dying her hair red',
                                        'A tattoo made by the woman regarding the decisions and preferences of the woman',
                                        'Regarding the decisions and preferences of the woman the laugh of the woman',
                                        'Regarding the decisions and preferences of the woman the desire of the woman to return to her country',
                                        'Other the womans desire to marry',
                                        'The womans meeting with her relatives regarding the womans decisions and preferences',
                                        'Regarding the womans decisions and preferences the woman helping the mans brother',
                                        'Regarding the womans decisions and preferences the womans alimony request',
                                        'Regarding the womans decisions and preferences the womans request for a car',
                                        'Regarding the womans decisions and preferences the womans request for a dress',
                                        'Regarding the womans decisions and preferences Womans tattooing the child',
                                        'The womans testimony against the womans decisions and preferences',
                                        'Regarding the womans decisions and preferences Woman coming home late',
                                        'Regarding the decisions and preferences of the woman Woman being offended',
                                        'Regarding the womans decisions and preferences the womans request for a summer house',
                                        'Woman misleads the tool regarding the decisions and preferences of the woman',
                                        'Regarding the womans decisions and preferences the womans buying new clothes',
                                        'Regarding the womans decisions and preferences the woman does not want to wear a headscarf']
    }
victims2['excuses_cat'] = victims2['excuse_en'].map(lambda x: next((k for k, v in remapping_excuses.items() if x in v), None))

Dropping old excuses column

In [36]:
victims2 = victims2.drop('excuse', axis=1)

In [37]:
victims2 = victims2.drop('excuse_en', axis=1)

In [38]:
victims2

,age,date,prov_name,perpetrator_en_new,excuses_cat
0,18.0,31/12/2017,istanbul,boyfriend,Unknown
1,17.0,31/12/2017,burdur,boyfriend,Separation & Jealousy
2,38.0,30/12/2017,bolu,boyfriend,Other
3,53.0,27/12/2017,ankara,boyfriend,Separation & Jealousy
4,22.0,25/12/2017,diyarbakir,partner,Separation & Jealousy
...,...,...,...,...,...
1959,24.0,14/01/2010,ankara,relative,Masculinity&Honor
1960,34.0,12/1/2010,adana,attacker,Separation & Jealousy
1961,57.0,12/1/2010,kutahya,brother,Unknown
1962,34.0,11/1/2010,adana,boyfriend,Unknown


### Check for nulls in victims data

In [39]:
victims2.isna().sum()

age                   70
date                   0
prov_name              3
perpetrator_en_new     0
excuses_cat            0
dtype: int64

We drop the observation that have no city, as we cannot include them in our model based on city demographics

In [40]:
victims2 = victims2[victims2['prov_name'].notna()]

In [41]:
victims2.isna().sum()

age                   70
date                   0
prov_name              0
perpetrator_en_new     0
excuses_cat            0
dtype: int64

### Check for types

In [42]:
victims2.dtypes

age                   float64
date                   object
prov_name              object
perpetrator_en_new     object
excuses_cat            object
dtype: object

In [43]:
victims2

,age,date,prov_name,perpetrator_en_new,excuses_cat
0,18.0,31/12/2017,istanbul,boyfriend,Unknown
1,17.0,31/12/2017,burdur,boyfriend,Separation & Jealousy
2,38.0,30/12/2017,bolu,boyfriend,Other
3,53.0,27/12/2017,ankara,boyfriend,Separation & Jealousy
4,22.0,25/12/2017,diyarbakir,partner,Separation & Jealousy
...,...,...,...,...,...
1959,24.0,14/01/2010,ankara,relative,Masculinity&Honor
1960,34.0,12/1/2010,adana,attacker,Separation & Jealousy
1961,57.0,12/1/2010,kutahya,brother,Unknown
1962,34.0,11/1/2010,adana,boyfriend,Unknown


Convert date

In [44]:
victims3 = victims2.copy()

In [45]:
victims3['date'] = pd.to_datetime(victims3['date'], format='%d/%m/%Y')

# 3. Check politics, and country trend to merge with demographic trends of regions

Check for Nulls, 0's and type of data.

The aim is to merge these with the regional data so that we can have an overview of regional trends as well as country trends. We will check for collinearity in the EDA notebook and, if needed, drop some features

In [46]:
demo_trends.isna().sum()

Year                     0
egalitarian_dem_index    0
wmn_civil_soc            0
acad_freedom             0
dtype: int64

In [47]:
demo_trends.isin([0]).sum()

Year                     0
egalitarian_dem_index    0
wmn_civil_soc            0
acad_freedom             0
dtype: int64

Demo_trends is clean and ready to merge, we drop some rows, as we only need data from 2010 to 2017.

In [48]:
demo_trends.drop(demo_trends[demo_trends['Year'] < 2010].index, inplace = True) 

In [49]:
demo_trends.drop(demo_trends[demo_trends['Year'] > 2017].index, inplace = True) 

In [50]:
elections.isna().sum()

prov_name     0
party_2009    0
perc_2009     0
party_2014    0
perc_2014     0
party_2019    0
perc_2019     0
dtype: int64

In [51]:
elections.isin([0]).sum()

prov_name     0
party_2009    0
perc_2009     0
party_2014    0
perc_2014     0
party_2019    0
perc_2019     0
dtype: int64

In [52]:
elections.dtypes

prov_name      object
party_2009     object
perc_2009     float64
party_2014     object
perc_2014     float64
party_2019     object
perc_2019     float64
dtype: object

For the elections data, we check which parties are included. 
Here is the list:
- AKP
- CHP
- MHP
- Ind.
- BDP(DBP)
- DTP
- DSP
- HDP
</br>


In [53]:
elections['party_2009'].unique()

array(['MHP', 'AKP', 'CHP', 'DTP', 'DSP', 'BBP', 'IND', 'DP'],
      dtype=object)

In [54]:
elections['party_2014'].unique()

array(['MHP', 'AKP', 'BDP', 'CHP'], dtype=object)

In [55]:
elections['party_2019'].unique()

array(['CHP', 'AKP', 'MHP', 'HDP', 'TKP'], dtype=object)

Based on https://en.wikipedia.org/wiki/List_of_political_parties_in_Turkey and https://www.cambridge.org/core/journals/new-perspectives-on-turkey/article/keeping-power-through-opposition-party-system-change-in-turkey/7D9211968313E58D1B28FEA68976E89F we will place the parties on a spectrum from -2 (extreme right) to 2 (extreme left).

- MHP Far-right
- AKP Right-wing
- Ind.Centre-left
- DSP Centre-left
- CHP Centre-left
- DTP Left-wing nationalism
- BDP(DBP) Left-wing
- HDP Left-wing

Until 2011, in Turkey, instead of using left–right
ideology, parties weree better scaled on their position in 
the religious–secular issue. The Turkish left wing differed from European undestanding of the left, having close ties with the military. (Aydogan, A., & Slapin, J. (2012). Left-Right Reversed: Parties and Ideology in Modern Turkey. SSRN Electronic Journal. doi: 10.2139/ssrn.2165409).
Nowaday, however, it can be argued that their positions changed drastically from what was presented in the article. For example, the AKP party their secular and liberal stance. Thus, we will scale parties as following (from 2-left-secular to -2-right-religious). This scaling wants to simplify an aspect that should have various dimension. For the sake of this research, we will keep only one for now:
- CHP 2
- AKP -2
- MHP -1
- DTP 0
- BDP(DBP) 0
- HDP 0
- Ind. 1

In [56]:
remapping_parties = {
        'AKP':int(-2),
        'CHP':int(2),
        'MHP':int(-1),
        'BBP':int(-1),
        'DTP':int(0),
        'DBP':int(0),
        'BDP':int(0),
        'HDP':int(0),
        'IND':int(1),
        'DSP':int(1),
        'DP':int(1),
        'TKP':int(2)
}
elections['party_2009'] = elections['party_2009'].replace(remapping_parties)

In [57]:
elections['party_2014'] = elections['party_2014'].replace(remapping_parties)

In [58]:
elections['party_2019'] = elections['party_2019'].replace(remapping_parties)

In [59]:
elections.dtypes

prov_name      object
party_2009      int64
perc_2009     float64
party_2014      int64
perc_2014     float64
party_2019      int64
perc_2019     float64
dtype: object

In [60]:
elections['party_2009'] =elections['party_2009'].astype(int) 

In [61]:
elections['party_2014'] =elections['party_2014'].astype(int) 

In [62]:
elections['party_2019'] =elections['party_2019'].astype(int) 

Lowercase column titles and name of cities

In [63]:
elections.columns=[e.lower().replace(' ', '_') for e in elections.columns] 

We do not care about the number of votes. What is relevant is just:
- the city
- the party who won (scaled from left  2 to right - 2)
- the percentage of votes by which the party won. This allows us to get a weighted results and a better idea of regional attitudes (always as comparison between left to right, secular to religious). We will multiply this by the election results, to get the weighted attitude for each of the three years.

- the lowest the number, the more the political attitude is tending toward the right.

In [64]:
elections['pol_2009'] = elections['party_2009'] * elections['perc_2009']

In [65]:
elections['pol_2014'] = elections['party_2014'] * elections['perc_2014']

In [66]:
elections['pol_2019'] = elections['party_2019'] * elections['perc_2019']

In [67]:
elections['prov_name'] = elections['prov_name'].str.lower()

In [68]:
elections_cleaned =  pd.concat([elections['prov_name'], elections['pol_2009'],elections['pol_2014'],elections['pol_2019']], axis=1)

In [69]:
elections_cleaned

,prov_name,pol_2009,pol_2014,pol_2019
0,adana,-29.7,-33.6,107.26
1,adiyaman,-98.6,-113.0,-106.44
2,afyonkarahisar,-96.4,-98.8,-85.92
3,agri,-79.2,0.0,-111.14
4,aksaray,-116.4,-113.8,-80.94
...,...,...,...,...
76,usak,-39.6,-86.6,-78.22
77,van,0.0,0.0,0.00
78,yalova,35.4,96.0,91.82
79,yozgat,-109.4,-112.4,-81.72


# 4. Merging
#### a.Counting victims total in country
Count total victims from victim file and add as column on the country demographics trends to be able to run a regression on count.

In [70]:
count = victims3.groupby([victims3['date'].dt.year.rename('year')]).agg({'count'})

In [71]:
#flatten the headers
count.columns =count.columns.map('_'.join)
count = count.reset_index()
count

,year,age_count,date_count,prov_name_count,perpetrator_en_new_count,excuses_cat_count
0,2010,239,248,248,248,248
1,2011,226,230,230,230,230
2,2012,180,182,182,182,182
3,2013,205,209,209,209,209
4,2014,258,266,266,266,266
5,2015,267,283,283,283,283
6,2016,247,258,258,258,258
7,2017,269,285,285,285,285


In [72]:
count = pd.concat([count['year'], count['prov_name_count']], axis=1, keys=["year", "count_fem"])

In [73]:
deaths_country = pd.merge(demo_trends, count, left_on='Year', right_on='year', how='inner').drop('Year', axis=1)

In [74]:
deaths_country

,egalitarian_dem_index,wmn_civil_soc,acad_freedom,year,count_fem
0,0.390,0.366,0.482,2010,248
1,0.391,0.376,0.482,2011,230
2,0.384,0.378,0.467,2012,182
3,0.327,0.385,0.378,2013,209
4,0.290,0.386,0.324,2014,266
5,0.269,0.349,0.294,2015,283
6,0.221,0.332,0.088,2016,258
7,0.201,0.275,0.057,2017,285


# 4. Merging
#### b. Count of victims per region
<br>
Count total victims per region from victim file and add as column on the region demographics to run regression. </br>

### Political Elections as proxi for regional attitude
Additionally, I want to merge the information I found for the regional elections and use them as a proxy for political attitude shifts.
</br>
<br>
The elections were not run in the same exact years of our dataset. Hence, I have just chosen 2009 (just before our dataset femicides cases) and 2019(just after), to be able to see such shifts.
</br>
As our dataset goes from 2010 to 2017, I will proceed with the following assignment.
- 2010 - 2012 attitude: election results from 2009
- 2013 - 2016 attitude: election results from 2014
- 2017 attitude: election results from 2019

#### Add Count of Victims

In [75]:
victims3

,age,date,prov_name,perpetrator_en_new,excuses_cat
0,18.0,2017-12-31,istanbul,boyfriend,Unknown
1,17.0,2017-12-31,burdur,boyfriend,Separation & Jealousy
2,38.0,2017-12-30,bolu,boyfriend,Other
3,53.0,2017-12-27,ankara,boyfriend,Separation & Jealousy
4,22.0,2017-12-25,diyarbakir,partner,Separation & Jealousy
...,...,...,...,...,...
1959,24.0,2010-01-14,ankara,relative,Masculinity&Honor
1960,34.0,2010-01-12,adana,attacker,Separation & Jealousy
1961,57.0,2010-01-12,kutahya,brother,Unknown
1962,34.0,2010-01-11,adana,boyfriend,Unknown


In [76]:
count1 = victims3.groupby([victims3['prov_name'], victims3['date'].dt.year.rename('year')]).agg({'count'})

In [77]:
#flatten the headers
count1.columns =count1.columns.map('_'.join)
count1 = count1.reset_index()
count1

,prov_name,year,age_count,date_count,perpetrator_en_new_count,excuses_cat_count
0,adana,2010,18,20,20,20
1,adana,2011,15,15,15,15
2,adana,2012,9,9,9,9
3,adana,2013,7,7,7,7
4,adana,2014,14,14,14,14
...,...,...,...,...,...,...
444,zonguldak,2013,3,3,3,3
445,zonguldak,2014,5,5,5,5
446,zonguldak,2015,1,1,1,1
447,zonguldak,2016,4,4,4,4


In [78]:
count_fem_prov = pd.concat([count1['prov_name'], count1['year'], count1['date_count']], axis=1, keys=["prov_name", "year", "count"])

Final clean up before merging. Some extra spaces were present in count_fem_prov and regions indexes had to be dropped.

In [79]:
count_fem_prov['prov_name'] = count_fem_prov['prov_name'].str.strip()

In [80]:
regions_info = regions_info.reset_index()

In [81]:
regions_info["count"]=""
for i in range(count_fem_prov.shape[0]):
    for j in range(regions_info.shape[0]):
        if regions_info.at[j, "prov_name"] == count_fem_prov.at[i, "prov_name"] and regions_info.at[j, "year"] == count_fem_prov.at[i, "year"]:
            regions_info.at[j, "count"] = count_fem_prov.at[i, "count"]        

drop index

In [82]:
regions_info = regions_info.drop('index', axis=1)

Check for nulls after adding count. Are there provinces with no count?

In [83]:
round(regions_info.isna().sum()/len(regions_info),4)*100 

prov_code           0.0
prov_name           0.0
year                0.0
all_wm_prov         0.0
knows_wm_prov       0.0
total_pop           0.0
ethnic_turkish      0.0
ethnic_kurdish      0.0
ln_casualty         0.0
ln_gdppc_cons       0.0
divorce_rate_ma5    0.0
mosques_perk        0.0
ceasefire           0.0
fm_diff_educ        0.0
count               0.0
dtype: float64

We are good to go!

#### Add Politics to demographics 
- see chapter 4b for explanation loop
- got the data from https://secim.haberler.com/2014/illerde-durum/

In [84]:
regions_info["pol_attitude"]=""
for i in range(elections_cleaned.shape[0]):
    for j in range(regions_info.shape[0]):
        if regions_info.at[j,"prov_name"] == elections_cleaned.at[i,"prov_name"]:
            if regions_info.at[j,"year"] in range(2010, 2013):
                regions_info.at[j, "pol_attitude"] =elections_cleaned.at[i,"pol_2009"]           
            elif regions_info.at[j,"year"] in range(2013, 2017):
                regions_info.at[j, "pol_attitude"] =elections_cleaned.at[i,'pol_2014']  
            elif regions_info.at[j,"year"]in range(2017, 2020):
                regions_info.at[j, "pol_attitude"] =elections_cleaned.at[i,"pol_2019"]  

Checking the count for empty values and spaces

In [85]:
regions_info['count'].unique()

array([20, 15, 9, 7, 14, 12, 11, 18, '', 1, 2, 4, 3, 10, 16, 13, 5, 6, 8,
       37, 27, 31, 28, 46, 34], dtype=object)

In [86]:
regions_info['count']= regions_info['count'].replace(r'^\s*$', np.nan, regex=True)

In [87]:
regions_info.to_csv('count')

,prov_code,prov_name,year,all_wm_prov,knows_wm_prov,total_pop,ethnic_turkish,ethnic_kurdish,ln_casualty,ln_gdppc_cons,divorce_rate_ma5,mosques_perk,ceasefire,fm_diff_educ,count,pol_attitude
0,1,adana,2010,20.0,19.0,2085225.0,82.03772,13.370917,3.178054,4.187798,1.458,0.606649,0,-0.062882,20.0,-29.70
1,1,adana,2011,15.0,15.0,2108805.0,82.03772,13.370917,3.178054,4.270533,1.532,0.599866,0,-0.068188,15.0,-29.70
2,1,adana,2012,9.0,9.0,2125635.0,82.03772,13.370917,3.178054,4.310695,1.632,0.640750,0,-0.067100,9.0,-29.70
3,1,adana,2013,7.0,7.0,2149260.0,82.03772,13.370917,3.178054,4.350712,1.712,0.651852,1,-0.066711,7.0,-33.60
4,1,adana,2014,14.0,14.0,2165595.0,82.03772,13.370917,3.178054,4.379060,1.790,0.654324,1,-0.067147,14.0,-33.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,81,duzce,2013,2.0,2.0,351509.0,87.50000,12.500000,0.000000,4.536124,1.454,1.894688,1,-0.104740,2.0,-109.00
644,81,duzce,2014,2.0,2.0,355549.0,87.50000,12.500000,0.000000,4.592223,1.572,1.881597,1,-0.103488,2.0,-109.00
645,81,duzce,2015,2.0,2.0,360388.0,87.50000,12.500000,0.000000,4.680219,1.730,1.856333,0,-0.102752,2.0,-109.00
646,81,duzce,2016,3.0,3.0,370371.0,87.50000,12.500000,0.000000,4.654947,1.850,1.833297,0,-0.095931,3.0,-109.00


Check again for percentage of nulls. Meaning we do not have a count for those rows. There can multiple causes:
- the cases may have not been recorded
- they are just missing in this dataset
- there were no cases
- cases were not reported (This has to be accounted for throughout our cases consideration)

As the model may be affected by considering the empty spaces as 0's, for now I will drop them.

In [88]:
round(regions_info.isna().sum()/len(regions_info),4)*100 

prov_code            0.00
prov_name            0.00
year                 0.00
all_wm_prov          0.00
knows_wm_prov        0.00
total_pop            0.00
ethnic_turkish       0.00
ethnic_kurdish       0.00
ln_casualty          0.00
ln_gdppc_cons        0.00
divorce_rate_ma5     0.00
mosques_perk         0.00
ceasefire            0.00
fm_diff_educ         0.00
count               32.41
pol_attitude         0.00
dtype: float64

In [89]:
regions_info=regions_info.dropna() 

#### Final Cleaning Regions

 We drop the province name and only keep the code

In [90]:
regions_info = regions_info.drop('prov_name', axis=1)

## Export Cleaned Files

We will continue only with regional trends for the regression and clustering. We will explore country data and victim levels data on Tableau

In [91]:
victims3.to_csv('victims_cleaned.csv')

In [92]:
regions_info.to_csv('regions_cleaned.csv')

In [93]:
deaths_country.to_csv('country_femicides.csv')